# Relationships Between World Diabetes Trends and Economic Changes  

## UNCC Data Analytics and Visualization - Project 1

### Fantastic Four - *Ayo, Mark, Neba, and Scott*


# Data Preparation Notebook

## Introduction

For this project the team compared four economic indicators with diabetes trends over the period 1990 to 2014.  The diabetes trends were from the [Non-Communicable Diseases Risk Factor Collaboration (NCD-RisC)](http://ncdrisc.org/about-us.html).  This group of health scientists working with the World Health Organization has published data on risk factors at [http://ncdrisc.org/data-downloads.html](http://ncdrisc.org/data-downloads.html).  The team used this data to generate diabetes trends for 217 countries.  Trend data from BMI and Blood pressure were also included, but in order to keep the scope under control, the team has focused on diabetes trends. 

Each team member selected a data set from the World Bank or United Nations to compare with diabetes trends.  The four indicators selected are shown below.

    * United Nations - Human Development index
    * World Bank - Number of Mobile Subscriptions per 100 people
    * World Bank - Access to electricity (% of population)
    * World Bank - CPIA property rights and rule-based governance rating (1=low to 6=high)


## Data Merging

The World Bank indicators were retrieved using their API.  The NCD-RisC and United nations data were downloaded as CSV files.

First we import some useful libraries.  A function, get_data, was created to automate the downloading of World Bank indicators.  It was saved in the external file, WBlib.py.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from WBlib import get_data

## Import NCD RisC Data

Health data for prevalence of diabetes, blood pressure and body mass index was downloaded from the [NCD RisC page](http://ncdrisc.org/data-downloads-diabetes.html).  This data was then merged into a single pandas dataframe. 


In [3]:
DM='data/NCD_RisC_Lancet_2016_DM_age_standardised_countries.csv'
BMI='data/NCD_RisC_Lancet_2017_BMI_age_standardised_countries.csv'
BP='data/NCD_RisC_Lancet_2016_BP_age_standardised_countries.csv'

DM=pd.read_csv(DM,encoding='iso-8859-1')
BMI=pd.read_csv(BMI,encoding='iso-8859-1')
BP=pd.read_csv(BP,encoding='iso-8859-1')

Health_Data = pd.merge(DM,BMI,on=['Country/Region/World','ISO','Sex','Year'])
Health_Data = Health_Data.merge(BP,on=['Country/Region/World','ISO','Sex','Year'])

## Import World Bank Data

__This pull from the World Bank can take a long time.__

Indicator ID codes can be found at the [World Bank data site](https://data.worldbank.org/indicator).

In [4]:


#"Scott - mobile phone subscriptions per 100 people"
indicator1 = "IT.CEL.SETS.P2"
#Ayo - Access to electricity (% of population)
indicator2 = "EG.ELC.ACCS.ZS"
#Neba - CPIA property rights and rule-based governance rating (1=low to 6=high)
indicator3 = "IQ.CPA.PROP.XQ"
#Choosing years from 1990:2014
years="1990:2014"

mobiles = get_data(indicator1, years, 'all')
electricity = get_data(indicator2, years, 'all')
governance = get_data(indicator3, years, 'all')

WBindicators = pd.merge(mobiles,electricity, on=['ISO','Year'])
WBindicators = WBindicators.merge(governance, on=['ISO','Year'])
WBindicators['Year'] = pd.to_numeric(WBindicators['Year'])
WBindicators.to_csv('data/indicators.csv')

__Start execution here with saved data to avoid long pull__

In [5]:
WBindicators = pd.read_csv('data/indicators.csv')

## Import UN Human Development Index Data

In order to merge the data with the World Bank indicator and health data, a ISO three letter code was added to each observation. 

Country data from the United Nations Human Development Index were assigned into quintiles for each year of the data.  a label was then added to each row indicating the quintile for that observation.  

In [6]:
hdi = pd.read_csv('data/Human development index (HDI).csv',encoding='iso-8859-1')

hdi['Country'] = hdi['Country'].apply(str.strip)
codes = pd.read_csv('data/countries_codes_and_coordinates.csv')
codes=codes[['Country', 'Alpha-3 code']]
hdi = hdi.merge(codes, on='Country')
hdi_years = [ '1990', '1991', '1992', '1993', '1994',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017']
hdi = hdi.melt(value_vars= hdi_years, id_vars=['Country','Alpha-3 code','HDI Rank (2017)'])
hdi.columns=['Country','ISO','HDIrank2017','Year','HDIvalue']

hdi_quint = {}

for y in hdi_years:
    #print(type(y))
    tmp = hdi.groupby(['Year']).get_group(y)
    tmp = tmp[['HDIvalue']]
    tmp.columns=[f'HDI {y}']
    hdi_quint[y] = tmp.sort_values(f'HDI {y}',ascending=True).quantile([0.2,0.4,0.6, 0.8])
    
for y in hdi_years:
    hdi_quint[y]['bin']= 10 * (hdi_quint[y].index) / 2

# Convert Dictionary to dataframe
hdi_quint = (pd.concat(hdi_quint, sort=False)
        .reset_index(level=1, drop=True)
        .rename_axis('Year')
        .reset_index()
)

HDI_quint_grp_df = hdi_quint.groupby(['bin']).sum().head(10)

# Flip the data frame back over to get the years as columns. 
HDI_quint_grp_df = HDI_quint_grp_df.reset_index()
HDI_quint_grp_df = HDI_quint_grp_df.set_index('bin').T

# Set up seq as the merge variable in the quint file.
HDI_quint_grp_df = HDI_quint_grp_df.reset_index()
HDI_quint_grp_df = HDI_quint_grp_df.rename_axis('seq',axis=0)

HDI_quint_grp_df.columns = ['Year in Text', 'Cut_1', 'Cut_2', 'Cut_3', 'Cut_4']

# Set up seq as the merge variable in the years file to use as colums in the quint file.
# I gave up on trying to change the text string that included years in the quint file.  
# I'm seriously embarrased at the round about way my code is getting the simplest stuff done.
hdi_years_tall_pd = pd.DataFrame(data = hdi_years, columns = ['Year'])
hdi_years_tall_pd = hdi_years_tall_pd.rename_axis('seq',axis=0)

# Merge years into the quint group file. 
HDI_quint_grp_df = HDI_quint_grp_df.merge(hdi_years_tall_pd, left_on='seq', right_on='seq', how = 'outer')

# Merge the quint group with the main dataset you created yesterday. 
hdi = hdi.merge(HDI_quint_grp_df, left_on='Year', right_on='Year', how = 'outer')

def binning(row):
    if   row[4] != 'NaN':
        if row[4] < row[6]:
            return 1
        elif  row[4] < row[7]:
            return 2
        elif  row[4] < row[8]:
            return 3
        elif  row[4] < row[9]:
            return 4
        elif  row[4] <= 1.0:
            return 5
        else:
            return 'NaN'
        
hdi['bins'] = hdi.apply(lambda row : binning(row),axis=1) 

hdi['Year'] = pd.to_numeric(hdi['Year'])
hdi['ISO'] = hdi['ISO'].apply(lambda x : x[2:5])

## Merge HDI and WB Datasets and Save to File

World Bank and United Nations data are merged into one economic data dataframe and saved to CSV.

Health data was split into Men and Women.  Blood Pressure and BMI data was dropped from each dataframe.  Each dataframe was saved to CSV.

Finally, health and economic dataframes are merged to produce complete data sets for men and women.

The dataframes are then saved to CSV for use in the analysis notebook.

In [7]:
economic_data = pd.merge(hdi,WBindicators,on=['ISO','Year'])
economic_data.head()

,Country,ISO,HDIrank2017,Year,HDIvalue,Year in Text,Cut_1,Cut_2,Cut_3,Cut_4,bins,Unnamed: 0,IT.CEL.SETS.P2,EG.ELC.ACCS.ZS,IQ.CPA.PROP.XQ
0,Afghanistan,AFG,168,1990,NaN,HDI 1990,0.4126,0.5804,0.6712,0.7448,NaN,24,0.000000,0.010000,NaN
1,Albania,ALB,68,1990,0.645,HDI 1990,0.4126,0.5804,0.6712,0.7448,3,49,0.000000,100.000000,NaN
2,Algeria,DZA,85,1990,0.577,HDI 1990,0.4126,0.5804,0.6712,0.7448,2,74,0.001814,98.271378,NaN
3,Andorra,AND,35,1990,NaN,HDI 1990,0.4126,0.5804,0.6712,0.7448,NaN,124,0.000000,100.000000,NaN
4,Angola,AGO,147,1990,NaN,HDI 1990,0.4126,0.5804,0.6712,0.7448,NaN,149,0.000000,11.397808,NaN


In [8]:
economic_data.to_csv('data/combined_economic_data.csv')

In [9]:
Health_Data_Men = Health_Data.groupby('Sex').get_group('Men')
Health_Data_Women = Health_Data.groupby('Sex').get_group('Women')

In [10]:
Health_Data_Men = Health_Data_Men[Health_Data_Men.columns[0:6]]
Health_Data_Women = Health_Data_Women[Health_Data_Women.columns[0:6]]

In [11]:
Health_Data_Women.head()

,Country/Region/World,ISO,Sex,Year,Age-standardised diabetes prevalence,Lower 95% uncertainty interval
7000,Afghanistan,AFG,Women,1980,0.053746,0.019543
7001,Afghanistan,AFG,Women,1981,0.054996,0.021399
7002,Afghanistan,AFG,Women,1982,0.056305,0.023432
7003,Afghanistan,AFG,Women,1983,0.057680,0.025645
7004,Afghanistan,AFG,Women,1984,0.059115,0.027713


In [12]:
Health_Data_Men.head()

,Country/Region/World,ISO,Sex,Year,Age-standardised diabetes prevalence,Lower 95% uncertainty interval
0,Afghanistan,AFG,Men,1980,0.044712,0.015339
1,Afghanistan,AFG,Men,1981,0.046114,0.016883
2,Afghanistan,AFG,Men,1982,0.047601,0.018745
3,Afghanistan,AFG,Men,1983,0.049173,0.020375
4,Afghanistan,AFG,Men,1984,0.050834,0.022269


In [14]:
all_data_men = Health_Data_Men.merge(economic_data,on=['ISO','Year'])
all_data_men = all_data_men[all_data_men.columns[1:]]
all_data_men.to_csv('data/all_data_men.csv')

all_data_women = Health_Data_Women.merge(economic_data,on=['ISO','Year'])
all_data_women = all_data_women[all_data_women.columns[1:]]
all_data_women.to_csv('data/all_data_women.csv')